In [7]:
import requests 
import json 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from time import process_time 

def compute_HLVVW(df):
    High = df['Price'].max()
    Low = df['Price'].min()
    Volume = df.Volume.iloc[-1]
    Close = df.Price.iloc[-1]
    df["VWAP"] = (Volume * ((High + Low + Close) /3)) / Volume
    return df

def tickRule(df):
    df['TickRule'] = 0
    for i in range(0,len(df)):                          
        if df.Price.iloc[i] > df.Price.iloc[i-1]:
            df.TickRule.iloc[i] = 1 
        elif df.Price.iloc[i] < df.Price.iloc[i-1]:
            df.TickRule.iloc[i] = -1 
        else: 
            df.TickRule.iloc[i] = df.TickRule.iloc[i-1]
    df = df.assign(TickRule = df['TickRule'].cumsum())
    return df
                    
# Choose to calculate volume as the division of the 
    # multiplication of the average ticks per day and their
        # respective bid-ask volumes average.
        # Considering that 80 bars per day in a normal day are filled by 2.3 million shares 
            # every 5 minutes.

# Following the same path, tick bars equals the division of 80 and the
    # division of the length of the data and the length of each day's data (tick-by-tick)
def thresholds(df,threshold):
    df = df.set_index('timestamp').astype(
        {'bid_vol':'int64', 'ask_vol':'int64','date':'datetime64'}).reset_index()
    total_ticks = len(df)
    Day_grp = df.groupby(['date'])

    Tick_per_bar = round((total_ticks / len(Day_grp)) / threshold)
    Vol_per_bar = round(((total_ticks / len(Day_grp))* (
        Day_grp.mean()['ask_vol'] + Day_grp.mean()['bid_vol']  / 2).mean())/ threshold)
    return [Tick_per_bar, Vol_per_bar]

def Variables(df):
    df =  df.assign(Volume = df['bid_vol'].cumsum() + df['ask_vol'].cumsum(),
                        Price = ((df['ask_vol'] * df['ask'] + df['bid_vol'] * df['bid'])
                                / (df['ask_vol'] + df['bid_vol'])),
                         MidPrice = (df['bid'] + df['ask'])/ 2,
                        Spread  = df['ask'] - df['bid'],
                       SpreadCm = (df['ask'] - df['bid']).cumsum()) 
    df = tickRule(df)

    #  The observed prices are the result of sequential trading against the bid-ask spread
    df = df.assign(ObservedPrice = df['Price'] + df['TickRule']*df['Spread'])
    df = df.set_index('timestamp').astype(
        {'bid_vol':'int64', 'ask_vol':'int64',
         'Volume':'int64', 'Price':'float64', 'ObservedPrice':'float64'}).reset_index()
    return df


def Volume_Bars(df):
    New_bar = Variables(df)
    Volume_Bars = pd.DataFrame(columns = ['timestamp', 'bid', 'ask', 'bid_vol', 
                                         'ask_vol', 'Volume','Price', 'MidPrice', 
                                         'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'])

#     Vol_per_bar = thresholds(data2,80)[1]# from instrument info table pull volperbar 
    
    Vol_per_bar = 5000000
    futureBar = pd.DataFrame(columns = ['timestamp','bid', 'ask', 'bid_vol','ask_vol'])
    
    while len(New_bar[New_bar['Volume'] >= Vol_per_bar]) > 1:
        i = New_bar[New_bar['Volume'] >= Vol_per_bar].index[0]
        Volume_Bars = Volume_Bars.append(compute_HLVVW(New_bar.iloc[:i]).iloc[-1])
        deprecatedBar = New_bar.iloc[:i]
        thirdQ = np.quantile(deprecatedBar['Volume'], .75).astype(int)
        futureBar = New_bar[New_bar['Volume'] >= thirdQ]
        futureBar = futureBar.drop([ 'Volume', 'Price', 'MidPrice',
                                  'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'],axis = 1)
        New_bar = Variables(futureBar)
    
    if (len(Volume_Bars) == 0) & (len(futureBar) == 0):
        futureBar = df
        
    return [Volume_Bars,futureBar]

def Tick_Bars(df):
    New_bar = Variables(df)
    Tick_Bars = pd.DataFrame(columns = ['timestamp', 'bid', 'ask', 'bid_vol', 
                                         'ask_vol', 'Volume','Price', 'MidPrice', 
                                         'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'])

#     Tick_per_bar = thresholds(data2,threshold)[1]# from instrument info table pull volperbar 
    Tick_per_bar = 629

    futureBar = pd.DataFrame(columns = ['timestamp','bid', 'ask', 'bid_vol','ask_vol'])
   
    while len(New_bar) > Tick_per_bar:
        i = New_bar[New_bar.index >= Tick_per_bar].index[0]
        Tick_Bars = Tick_Bars.append(compute_HLVVW(New_bar.iloc[:i]).iloc[-1])
        deprecatedBar = New_bar.iloc[:i]
        thirdQ = np.quantile(deprecatedBar.index[-1], .75).astype(int)
        futureBar = New_bar[New_bar.index >= thirdQ]
        futureBar = futureBar.drop([ 'Volume', 'Price', 'MidPrice',
                                  'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'],axis = 1)
        New_bar = Variables(futureBar)
        
    if (len(Tick_Bars) == 0) & (len(futureBar) == 0):
        futureBar = df
        
    return [Tick_Bars,futureBar]

In [8]:
# instruments ID 
# Api to get the list of instruments 
response2 = requests.get('https://freeserv.dukascopy.com/2.0/?path=api/instrumentList',
                        params= {'key' : 'rup1doorqo000000'})

# pd.DataFrame(response2.json()).iloc[0,1]
FAANG = {"Apple":"70002" ,"Facebook":"70094","Amazon":"70022","Google":"70118","Netflix":"70178"}



In [ ]:
# for loop for each day of 2019 
# Start the stopwatch / counter  
t1_start = process_time()  

instruments = FAANG["Netflix"]
# # empty dataframe with columns: ['timestamp','bid', 'ask', 'bid_vol','ask_vol']
# la3 = pd.DataFrame(columns = ['timestamp','bid', 'ask', 'bid_vol','ask_vol'])

# vols =  pd.DataFrame(columns = ['timestamp', 'bid', 'ask', 'bid_vol', 
#                                      'ask_vol',  'Volume','Price', 'MidPrice', 
#                                      'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'])

# ticks =  pd.DataFrame(columns = ['timestamp', 'bid', 'ask', 'bid_vol', 
#                                      'ask_vol', 'Volume','Price', 'MidPrice', 
#                                      'Spread', 'SpreadCm', 'TickRule', 'ObservedPrice'])

# futureBarTick = pd.DataFrame(columns = ['timestamp','bid', 'ask', 'bid_vol','ask_vol'])
# futureBarVolume = pd.DataFrame(columns = ['timestamp','bid', 'ask', 'bid_vol','ask_vol'])

# for each number between 199 - 150 (part of the 365 days of the year) going in negative way
for i in list(range(102,0,-1)):
    # substract from the 01-01-2020 the days that i equals
    date = datetime(2020, 1, 1) - timedelta(days=i)

    # converts date into timestamp and adds 9 hs and 30 minutes, 
        # and multiplies * 1000 to get miliseconds, while using int() to convert to integer
    start = int(datetime.timestamp(date + timedelta(hours = 9, minutes = 30)) * 1000)
    end = int(datetime.timestamp(date + timedelta(hours = 9, minutes = 35)) * 1000)

#     if len(pd.DataFrame(pd.DataFrame(requests.get('https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices', 
#                                 params={'key' : 'rup1doorqo000000','instrument':instruments,
#                                         'timeFrame':'tick','count':'5000',
#                                         'start':start, 'end':end}).json())['ticks'].tolist())) > 0:
#         # while the datetime is less than 15:56 hs
    while datetime.fromtimestamp(end/1000).time() < time(hour = 15, minute = 56):

        # set parameter conditions, the key here is that start and end are always different
        parameters = {'key' : 'rup1doorqo000000','instrument':instruments,'timeFrame':'tick','count':'5000',
           'start':start, 'end':end}

        # api pull request
        response = requests.get('https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices', 
                                params=parameters) 

        # convert pull request into json file
        la = pd.DataFrame(response.json())

        # gets tick data from json file and converts it to list and then df
        pe = pd.DataFrame(la['ticks'].tolist())

        la3 = pd.concat([la3,pe])

        for k in range(len(pe)):
            pe['timestamp'].iloc[k] = datetime.fromtimestamp(pe['timestamp'].iloc[k]/1000)

#             pe = pe.assign(date = [d.date() for d in pe['timestamp']])

        vbars = Volume_Bars(pd.concat([futureBarVolume,pe]))
        tbars = Tick_Bars(pd.concat([futureBarTick,pe]))

        futureBarVolume = vbars[1]
        futureBarTick = tbars[1]

        vols = pd.concat([vols, vbars[0]])
        ticks = pd.concat([ticks, tbars[0]])

        # adds five minutes to both start and end of the pull request
        start = int(datetime.timestamp(datetime.fromtimestamp(start/1000) + timedelta(minutes = 5)) * 1000)
        end = int(datetime.timestamp(datetime.fromtimestamp(end/1000) + timedelta(minutes = 5)) * 1000)

# converts timetstamp to datetime and substracts 5 vs
la3['timestamp'] = pd.to_datetime(la3['timestamp'], unit='ms') - timedelta(hours= 5)
la3 = la3.assign(date = [d.date() for d in la3['timestamp']])
la3['Instrument'] = "Netflix"

# Stop the stopwatch / counter 
t1_stop = process_time() 

print("Elapsed time:", t1_stop, t1_start)  
print("Elapsed time during the whole program in seconds:", 
                                     t1_stop-t1_start)
la3.to_csv("Netflix" + "MainData.csv")
ticks['Instrument'] = "Netflix"
vols['Instrument'] = "Netflix"
ticks.to_csv("Netflix" + "ticks.csv")
vols.to_csv("Netflix"+ "vols.csv")



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [13]:
la3.to_csv("Netflix" + "MainData.csv")
ticks['Instrument'] = "Netflix"
vols['Instrument'] = "Netflix"
ticks.to_csv("Netflix" + "ticks.csv")
vols.to_csv("Netflix"+ "vols.csv")


In [16]:
for i in list(range(363,0,-1)):
    # substract from the 01-01-2020 the days that i equals
    print(i,datetime(2020, 1, 1) - timedelta(days=i))

363 2019-01-03 00:00:00
362 2019-01-04 00:00:00
361 2019-01-05 00:00:00
360 2019-01-06 00:00:00
359 2019-01-07 00:00:00
358 2019-01-08 00:00:00
357 2019-01-09 00:00:00
356 2019-01-10 00:00:00
355 2019-01-11 00:00:00
354 2019-01-12 00:00:00
353 2019-01-13 00:00:00
352 2019-01-14 00:00:00
351 2019-01-15 00:00:00
350 2019-01-16 00:00:00
349 2019-01-17 00:00:00
348 2019-01-18 00:00:00
347 2019-01-19 00:00:00
346 2019-01-20 00:00:00
345 2019-01-21 00:00:00
344 2019-01-22 00:00:00
343 2019-01-23 00:00:00
342 2019-01-24 00:00:00
341 2019-01-25 00:00:00
340 2019-01-26 00:00:00
339 2019-01-27 00:00:00
338 2019-01-28 00:00:00
337 2019-01-29 00:00:00
336 2019-01-30 00:00:00
335 2019-01-31 00:00:00
334 2019-02-01 00:00:00
333 2019-02-02 00:00:00
332 2019-02-03 00:00:00
331 2019-02-04 00:00:00
330 2019-02-05 00:00:00
329 2019-02-06 00:00:00
328 2019-02-07 00:00:00
327 2019-02-08 00:00:00
326 2019-02-09 00:00:00
325 2019-02-10 00:00:00
324 2019-02-11 00:00:00
323 2019-02-12 00:00:00
322 2019-02-13 0

In [17]:
vols

,timestamp,bid,ask,bid_vol,ask_vol,Volume,Price,MidPrice,Spread,SpreadCm,TickRule,ObservedPrice,VWAP,Instrument
887,2019-01-03 09:32:11.348,143.861,143.903,320,7500,4996890,143.901281,143.8820,0.042,48.643,-4,143.733281,144.106850,Netflix
983,2019-01-03 09:34:09.836,144.157,144.203,10,7500,4993631,144.202939,144.1800,0.046,52.224,-30,142.822939,144.109951,Netflix
937,2019-01-03 09:35:50.859,143.387,143.423,110,37,4993135,143.396061,143.4050,0.036,46.914,-20,142.676061,143.663894,Netflix
985,2019-01-03 09:37:30.378,143.148,143.183,10,7500,4990689,143.182953,143.1655,0.035,47.246,10,143.532953,143.241480,Netflix
972,2019-01-03 09:39:19.218,143.288,143.332,110,4,4987916,143.289544,143.3100,0.044,44.905,25,144.389544,143.222261,Netflix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,2019-09-20 14:08:51.249,267.197,267.283,1000,1000,4998000,267.240000,267.2400,0.086,320.632,-25,265.090000,267.010000,Netflix
2498,2019-09-20 14:38:03.559,268.858,268.952,1000,1000,4998000,268.905000,268.9050,0.094,230.222,-43,264.863000,268.281667,Netflix
2498,2019-09-20 15:05:22.845,270.118,270.192,1000,1000,4998000,270.155000,270.1550,0.074,233.354,79,276.001000,269.251667,Netflix
2498,2019-09-20 15:34:07.129,269.307,269.423,1000,1000,4998000,269.365000,269.3650,0.116,210.862,25,272.265000,269.578333,Netflix
